# Fysikklab, gruppe 206, team 3
___Vetle Almås, Oda Vinje Lind, Nadia Andersen Sayed, Marte Stürtzel gulbrandsen___

Følgende notebook genererer en rullebane basert på koordinatene til åtte festepunkter. 
Baneformen $y(x)$ beregnes med __CubicSpline__ fra interpolate-biblioteket i SciPy. 

Først kommer kodene som er nødvednige for å plotte figuren med våre egne verdier. Her setter vi inn festepunktene for berg-og-dal-bannen vår som er:

<par> $[0.263, 0.212, 0.206, 0.169, 0.181, 0.168, 0.169, 0.118]$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CubicSpline

h = 0.200
xfast=np.asarray([0,h,2*h,3*h,4*h,5*h,6*h,7*h])

xmin = 0.000
xmax = 1.401
dx = 0.001
ymax = 3000

yfast = []
yfast_m = [0.263, 0.212, 0.206, 0.169, 0.181, 0.168, 0.169, 0.118] #yfast i meter

for i in yfast_m:
    j = i/1000
    yfast.append(int(j)) #ny liste med yfast i mm

inttan = np.diff(yfast)/h
cs = CubicSpline(xfast, yfast, bc_type='natural')

xmin = 0.000
xmax = 1.401
dx = 0.001

x = np.arange(xmin, xmax, dx) 

Nx = len(x)
y = cs(x)       
dy = cs(x,1)    
d2y = cs(x,2)   

# Her kommer våre egne beregninger

Først velger vi baneformen og finner fart ved å sette inn funksjonen, $v(x) = \sqrt\frac{2g(y_{0} - y(x))}{1 + c}$. Verdier fra teoriarket:

<par><li> $c = 2 / 5 $
    
<par><li> $M = 31 [g] = 0.031 [kg]$
    
<par><li> $R = 11 [mm] = 0.011 [m]$
    
<par><li> $g = 9.81 [m/s^2]$
    
`v_x` tilsvarer hastigheten i x-retning <br>

In [ ]:
g = 9.81
M = 0.0304
R = 0.011
c = 2/5
#Io = c*M*R**2

v_x = np.sqrt(2*g*(y[0]-y)/(1+c))

Deretter ønsker vi å definere krumningen, $κ = \frac{y''}{[1 + (y')^2]^{3/2}}$, og sentripetalakselerasjonen, $a⊥  = v^2κ$.  Vi finner helningsvinkelen, $β$ ved å ta $\frac{dy}{dx}$ gjennom numpy-funksjonen $np.arctan()$. Krumning og sentripetalakselerasjon brukes i ligningen for å finne normalkraften, $N$. 
Ligningen for normalkraft kommer fra:

<par><li> $N − Mg cos β = M a⊥$
    
`k` tilsvarer krumningen <br>
`a_s` tilsvarer sentripetalakselerasjonen <br>
`beta` tilsvarer helningsvinkelen <br>
`f` tilsvarer friksjonen <br>
`N` tilsvarer normalkraften <br>
`fN` tilsvarer absoluttverdien av friksjonen delt på normalkraften <br>

In [ ]:
k = d2y/(1+((dy**2))**(3/2))
a_s = (v_x**2)*k
beta = np.arctan(dy)

f = (2*M*g*np.sin(beta))/7
N = M*(g*np.cos(beta)+a_s)
fN = np.abs(f/N)

For å finne tiden som kulen bruker bortover x-aksen så ønsker man å finne fartskomponenten i x-retning. Dette gjør vi ved å gange arrayen `Vx` med $cos β$. Vi lager en array med kun nullere. Deretter itererer vi over denne og setter $V_{ax} = {(V_{x}[i+1] + V_x[i])}* \frac{1}{2}$. For å finne tiden lager vi en ny liste med nullere. Vi lager så en ny array, $∆t = \frac{∆x}{V_{ax}}$ og itererer over denne ved å setter $t = t + ∆t$. Vi får da ut en array som viser hvor lang tid den har brukt til et gitt punkt på x-aksen.

`Vx` tilsvarer banefarten <br>
`V_ax`tilsvarer gjennomsnittsfarten mellom to punkt i x-retning <br>
`dt` tilsvarer tiden kulen har brukt på intervallet <br>
`t` tilsvarer tiden brukt til neste punkt på x-aksen <br>

In [ ]:
Vx = v_x * np.cos(beta)
V_ax = np.zeros(Nx-1)
for i in range(Nx-1):
    V_ax[i] = 0.5*(Vx[i+1]+Vx[i])
    
dt = dx/V_ax #brukes denne slik som teksten sier??

t = np.zeros(Nx)
for i in range(Nx-1):
    t[i+1] = t[i]+dt[i]

__Baneform:__

In [ ]:
baneform = plt.figure('y(x)',figsize=(12,6))
plt.plot(x,y,xfast,yfast,'*')
plt.title('Banens form')
plt.xlabel('$x$ (m)',fontsize=20)
plt.ylabel('$y(x)$ (m)',fontsize=20)
plt.ylim(0.0,0.40)
plt.grid()
plt.show()

print('Starthøyde:',yfast[0]*100,'cm')
print('Festepunkthøyder(m)',yfast)

Banens __helningsvinkel__ $β$ overstiger ikke ...◦
i absoluttverdi:

In [ ]:
helningsvinkel = plt.figure('v(x)',figsize=(12,6))
plt.plot(x, beta)
plt.title('Helningsvinkel')
plt.xlabel('$x$ (m)',fontsize=20)
plt.ylabel('$β(grader)$',fontsize=20)
plt.grid()
plt.show()

Banens __krumning__ ligger mellom ... og ..., slik at minste krumningsradius er ca. ... .

In [ ]:
krumming = plt.figure('v(x)',figsize=(12,6))
plt.plot(x, k)
plt.title('Krumning')
plt.xlabel('$x$ (m)',fontsize=20)
plt.ylabel('$K(x)(1/m)$',fontsize=20)
plt.grid()
plt.show()

Med en kompakt kule ($c = \frac{2}{5}$) og $m = 30.4 g$ blir __fartsgrafen:__

In [ ]:
fart = plt.figure('v(x)',figsize=(12,6))
plt.plot(x, v_x)
plt.title('Fart som funksjon av x')
plt.xlabel('$x$ (m)',fontsize=20)
plt.ylabel('$v$ (m/s)',fontsize=20)
plt.grid()
plt.show()

__Normalkraften:__

In [ ]:
normalkraft = plt.figure('v(x)',figsize=(12,6))
plt.plot(x, N/M*g)
plt.title('Normalkraft')
plt.xlabel('$x$ (m)',fontsize=20)
plt.ylabel('$N/Mg$',fontsize=20)
plt.grid()
plt.show()

__Forholdet mellom friksjonskraften og normalkraften__ overstiger ikke verdeien ...:

In [ ]:
friksjon= plt.figure('v(x)',figsize=(12,6))
plt.plot(x, fN)
plt.title('Forhold mellom friksjonskraft og normalkraft')
plt.xlabel('$x$ (m)',fontsize=20)
plt.ylabel('$|f/N|$',fontsize=20)
plt.grid()
plt.show()

__Horistontal posisjon som funksjon av tid:__

In [ ]:
tid = plt.figure('v(x)',figsize=(12,6))
plt.plot(t, x)
plt.title('Horisontal posisjon som funksjon av tiden')
plt.xlabel('$t$ (s)',fontsize=20)
plt.ylabel('$x$ (m)',fontsize=20)
plt.grid()
plt.show()

print('Vi ser at reisen tok ca.', t , 'sekunder.') #legge inn t

__Hastighet v som funksjon av tiden t:__

In [ ]:
fartvedtid = plt.figure('v(x)',figsize=(12,6))
plt.plot(t, v_x)
plt.title('Hastighet som funksjon av tiden')
plt.xlabel('$t$ (s)',fontsize=20)
plt.ylabel('$v$ (m/s)',fontsize=20)
plt.grid()
plt.show()